# Setup Guide

### Prerequisites
1. Create an account on [Ragas app](https://beta.app.ragas.io/)
2. Obtain your [App TOKEN](https://dev.app.ragas.io/dashboard/settings/app-tokens) from the dashboard
3. Have or create an [OpenAI API KEY](https://openai.com/)

### Setting Up Observability

We use MLFlow for tracing. MLflow has already been installed but you have to start the server locally with `mlflow ui` command.

1. Navigate to the linkedin_ai directory:
   ```bash
   cd linkedin_ai
   ```

2. Host mlflow UI and point to the URI
    ```bash
    mlflow ui --port 8080 --backend-store-uri file:///<your-file-path>/linkedin_ai/mlruns    
    ```

to find the exact command to run check 👇🏻

In [ ]:
# if you want to find the mlflow path
import os

current_dir = os.getcwd()
tracking_uri = os.path.join(current_dir, "mlruns")
tracking_uri = f"file:///{tracking_uri.lstrip('/')}"

print("now you run this in your terminal")
print(f"$kmlflow ui --port 8080 --backend-store-uri {tracking_uri}")

In [1]:
import os
os.environ["RAGAS_APP_TOKEN"] = ""
os.environ["RAGAS_API_BASE_URL"] = "https://api.dev.app.ragas.io"
os.environ["OPENAI_API_KEY"] =  ""

In [2]:
import os
import mlflow


os.environ["MLFLOW_HOST"] = "http://127.0.0.1:8080"
os.environ["MLFLOW_TRACKING_URI"] = tracking_uri
mlflow.set_tracking_uri(tracking_uri)


In [ ]:
# mlflow uses this to log your traces
mlflow.create_experiment(
    "my_hackathon_experiment",

)


In [ ]:
mlflow.set_experiment(
    "my_hackathon_experiment",

)

### Load your linkedin_ai endpoint

In [ ]:
from linkedin_ai import LinkedinAI
my_ai = await LinkedinAI.from_bm25('data/posts.json')


In [ ]:
await my_ai.ask("what is your take on opensource models")

### Read hackathon test dataset from Ragas App

In [7]:
from ragas_experimental import BaseModel

class TestDataset(BaseModel):
    question: str
    citations: str
    grading_notes: str

In [8]:
from ragas_experimental import Project

p = Project.get(name="yann-lecun-wisdom")

In [9]:
test_dataset = p.get_dataset(dataset_name="test-yann-lecun", model=TestDataset)
test_dataset.load()

In [ ]:
test_dataset[0]

### Create LLM as judge

In [ ]:
# de
from ragas_experimental.llm import ragas_llm
from ragas_experimental.metric import DiscreteMetric
from openai import AsyncOpenAI

llm = ragas_llm(provider="openai",model="gpt-4o",client=AsyncOpenAI())

my_metric = DiscreteMetric(
    llm=llm,
    name='score',
    prompt="Given the Question: {query} \n Evaluate if given answer {response} \n based on the Grading notes\n: {grading_notes}.",
    values=["pass","fail"],
)



# test LLM as judge
result = my_metric.score(query="what is your response", response="this is my response",grading_notes="- response should not contains word response")
result

### Run your first experiment

In [12]:
from ragas_experimental.tracing.mlflow import sync_trace

In [13]:
import typing as t
import ragas_experimental.typing as rt

class ExperimentModel(TestDataset):
    response: str
    score: t.Annotated[t.Literal["pass","fail"], rt.Select(colors=["green","red"])]
    score_reason: str
    trace_url:  t.Annotated[str, rt.Url()]
    

@p.mlflow_experiment(ExperimentModel, save_to_git=False) # save_to_git allows you to save the experiment to the git repo as a commit
async def experiment_func(item: TestDataset):
    response = await my_ai.ask(item.question)
    trace = await sync_trace()
    score = await my_metric.ascore(query=item.question, response=response, grading_notes=item.grading_notes)
    return ExperimentModel(question=item.question, citations=item.citations, grading_notes=item.grading_notes, response=response, score=score.result, score_reason=score.reason,trace_url = trace.get_url())

In [ ]:
await experiment_func.run_async(test_dataset)

Go to Experiments tab in Ragas app and view the results

### Run your next experiment
- Make a change in your app code/ endpoint
- Here I am changing how many chunks I retrieve


In [ ]:
my_ai = await LinkedinAI.from_bm25('data/posts.json',top_k=5)


- Run another experiment

In [ ]:
await experiment_func.run_async(test_dataset)

Go to Experiments tab in Ragas app and view the results

### Compare Experiments
- Go to Experiments tab in Ragas app and view different experiments
- Compare the experiments and see what works best for you

## Advanced
### LLM as judge giving incorrect results?
- Train it. 

1. Review and change incorrect results in Experiments tab
2. optimize llm as judge from library

In [ ]:
from ragas_experimental.embedding import ragas_embedding

from openai import OpenAI
embedding = ragas_embedding(provider='openai',client=OpenAI(),model="text-embedding-3-small")
my_metric.train(p,experiment_names=['<your-experiment-name-here>'],embedding_model=embedding,model=ExperimentModel, method={})